# Analyzing RCT reemployment experiment

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data. 
  

In [1]:
# install.packages("librarian", quiet = T)
librarian::shelf(tidyverse, lmtest, sandwich, broom, hdm, quiet = T)
## loading the data
Penn <- as_tibble(read.table("https://github.com/d2cml-ai/14.388_r/raw/main/Data/penn_jae.dat", header=T ))
n <- dim(Penn)[1]
p_1 <- dim(Penn)[2]
Penn <- filter(Penn, tg == 4 | tg == 0)# subset(Penn, tg==4 | tg==0)
attach(Penn)

In [2]:
T4 <- (Penn$tg==4)
summary(T4)

   Mode   FALSE    TRUE 
logical    3354    1745 

In [3]:
glimpse(Penn)

Rows: 5,099
Columns: 23
$ abdt       <int> 10824, 10824, 10747, 10607, 10831, 10845, 10831, 10859, 105…
$ tg         <int> 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0,…
$ inuidur1   <int> 18, 1, 27, 9, 27, 27, 9, 27, 15, 28, 12, 22, 18, 1, 7, 18, …
$ inuidur2   <int> 18, 1, 27, 9, 27, 27, 9, 27, 15, 11, 12, 22, 18, 1, 7, 18, …
$ female     <int> 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,…
$ black      <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,…
$ hispanic   <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,…
$ othrace    <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ dep        <int> 2, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0,…
$ q1         <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ q2         <int> 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,…
$ q3         <int> 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,…
$ q4         <in

### Model 
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicators,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, if the RCT assumptions hold rigorously.


We also consider interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$

where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, if the RCT assumptions hold rigorously.

Under RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.

### Analysis

We consider 

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

and carry out robust inference using the *estimatr* R packages. 

## Carry out covariate balance check

This is done using "lm_robust" command which unlike "lm" in the base command automatically does the correct Eicher-Huber-White standard errors, instead othe classical non-robus formula based on the homoscdedasticity command.

In [4]:

m <- lm(T4 ~ (female + black + othrace + factor(dep) + q2 +q3 + q4 +
        q5 + q6 + agelt35 + agegt54 + durable + lusd + husd)^2, data = Penn)
coeftest(m, vcov = vcovHC(m, type="HC1"))



t test of coefficients:

                        Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)           0.32145725  0.16125607  1.9935 0.0462656 *  
female                0.10423328  0.13624779  0.7650 0.4442914    
black                 0.07164803  0.08288534  0.8644 0.3873969    
othrace               0.02801517  0.40496815  0.0692 0.9448502    
factor(dep)1         -0.07363340  0.20094637 -0.3664 0.7140574    
factor(dep)2         -0.10854072  0.15754307 -0.6890 0.4908810    
q2                   -0.02667937  0.16255836 -0.1641 0.8696419    
q3                   -0.00567387  0.16218178 -0.0350 0.9720934    
q4                    0.04334425  0.16233956  0.2670 0.7894821    
q5                    0.09386458  0.16157184  0.5809 0.5613028    
q6                   -0.22156423  0.15984049 -1.3862 0.1657604    
agelt35              -0.10923976  0.13323486 -0.8199 0.4123101    
agegt54              -0.43668630  0.13581268 -3.2154 0.0013111 ** 
durable              -0.12500967  0.

We see that that even though this is a randomized experiment, balance conditions are failed.

## Model Specification

In [5]:
# model specifications


# no adjustment (2-sample approach)
formula_cl <- log(inuidur1) ~ T4

# adding controls
formula_cra <- log(inuidur1) ~ T4 + (female + black + othrace + factor(dep) +
              q2 + q3 + q4 + q5 + q6 + agelt35 +agegt54 + durable +lusd + husd )^2
# Omitted dummies: q1, nondurable, muld


ols.cl <- lm(formula_cl, data = Penn)
ols.cra <- lm(formula_cra, data = Penn)


ols.cl = coeftest(ols.cl, vcov = vcovHC(ols.cl, type="HC1"))
ols.cra = coeftest(ols.cra, vcov = vcovHC(ols.cra, type="HC1"))

tidy(ols.cl)
tidy(ols.cra)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.05682971,0.02095476,98.155713,0.00000000
T4TRUE,-0.08545541,0.03585569,-2.383315,0.01719391


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.63308058,0.36809947,7.1531767,9.706770e-13
T4TRUE,-0.07968012,0.03559092,-2.2387766,2.521432e-02
female,-0.11460926,0.31639042,-0.3622400,7.171880e-01
black,-0.89902649,0.24024626,-3.7421040,1.845347e-04
othrace,-2.62692161,0.45415507,-5.7841953,7.729399e-09
factor(dep)1,-0.71969778,0.58121824,-1.2382574,2.156788e-01
factor(dep)2,-0.04055836,0.35983953,-0.1127124,9.102631e-01
q2,-0.15966999,0.37043166,-0.4310376,6.664596e-01
q3,-0.53989061,0.37007942,-1.4588507,1.446691e-01


The interactive specificaiton corresponds to the approach introduced in Lin (2013).

In [6]:

#interactive regression model;

X = model.matrix ( ~ (female + black + othrace + factor(dep) + 
                      q2 + q3 + q4 + q5 + q6 + agelt35 + agegt54 + durable + lusd + husd)^2)[,-1]

dim(X)

demean<- function(x){ x - mean(x)}

X = apply(X, 2, demean)

ols_ira = lm(log(inuidur1) ~ T4*X) 
ols_ira= coeftest(ols_ira, vcov = vcovHC(ols_ira, type="HC1"))
tidy(ols_ira)

[1] 5099  119

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.05761308,0.02077237,99.05527109,0.000000e+00
T4TRUE,-0.07550055,0.03560489,-2.12051040,3.401317e-02
Xfemale,-0.66628233,0.40902136,-1.62896709,1.033843e-01
Xblack,-0.86348620,0.29766982,-2.90081876,3.738472e-03
Xothrace,-3.81768810,0.93891012,-4.06608473,4.856015e-05
Xfactor(dep)1,0.03592643,0.64926578,0.05533393,9.558747e-01
Xfactor(dep)2,0.21175557,0.45232670,0.46814740,6.397000e-01
Xq2,-0.25464364,0.45645285,-0.55787502,5.769552e-01
Xq3,-0.62123263,0.45607674,-1.36212303,1.732217e-01


Next we try out partialling out with lasso

In [7]:
# library(hdm)

T4 = demean(T4)

DX = model.matrix(~T4 * X)[ ,-1]

rlasso.ira = summary(rlassoEffects(DX, log(inuidur1), index = 1))

print(rlasso.ira)

[1] "Estimates and significance testing of the effect of target variables"
   Estimate. Std. Error t value Pr(>|t|)  
T4  -0.07889    0.03555  -2.219   0.0265 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1




## Results

In [12]:
# glimpse(ols_ira)
ols_ira[2,1]

[1] -0.07550055

In [16]:
# library(xtable)
table<- matrix(0, 2, 4)
table[1,1]<-  ols.cl[2,1]
table[1,2]<-  ols.cra[2,1]
table[1,3]<-  ols_ira[2,1]
table[1,4]<-  rlasso.ira[[1]][1]

table[2,1]<-  ols.cl[2,2]
table[2,2]<-  ols.cra[2,2]
table[2,3]<-  ols_ira[2,2]
table[2,4]<-  rlasso.ira[[1]][2]


colnames(table)<- c("CL","CRA","IRA", "IRA w Lasso")
rownames(table)<- c("estimate", "standard error")
# tab<- xtable(table, digits=5)
table


,CL,CRA,IRA,IRA w Lasso
estimate,-0.08545541,-0.07968012,-0.07550055,-0.07888608
standard error,0.03585569,0.03559092,0.03560489,0.03555130


Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.


